In [1]:
%load_ext autoreload
%autoreload 2

### The following notebook is to run an experiment for Shot Boundary Detection

Based of off ideas presented in https://arxiv.org/pdf/1705.08214.pdf



### Imports

In [2]:
import time
import os
import copy
import random
import re

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import models, transforms
from torch.utils.tensorboard import SummaryWriter

#### Python files imported

- model_zoo.py: Contains all the models
- training.py: Contains training and validation code
- data.py: Contains all the datasets

In [3]:
from model_zoo import sbd_detector_1, sbd_detector_2
from training import train_step, val_step, train
from data import get_random_data

### Set seeds and get GPU details

In [4]:
seed = 8964
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.deterministic = True
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print("Current Device - %s" % device)

if torch.cuda.is_available():
    print("CUDA Device Count - %s" % torch.cuda.device_count())
    print("CUDA Device Name - %s" % torch.cuda.get_device_name())
    print("CUDA Device Memory - %0.2f GB"%(float(torch.cuda.get_device_properties(0).total_memory)/1024**3))

Current Device - cpu


## Choose the data

Options are get_random_data()

In [5]:
n_batches = 2 # No of batches
batch_size = 128
T = 10 # Time: Length of base sequence passed
N = 3 # Additional frames to be included in the sequence

random_dataloader = get_random_data(n_batches, batch_size, T, N)

## Choose the model

Options are sbd_detector_1, sbd_detector_2

- SBD_Detector_1 works for only base length frame sequence T
- SBD_Detector_2 works for variable length sequence (T+N). 


In [7]:
model = sbd_detector_2()
model.summarize(random_dataloader[0][0])
model = model.to(device)

Class: sbd_detector_2
Input Size: torch.Size([128, 3, 13, 64, 64])
After Layer 1: torch.Size([128, 16, 11, 30, 30])
After Layer 2: torch.Size([128, 24, 9, 14, 14])
After Layer 3: torch.Size([128, 32, 7, 6, 6])
After Layer 4: torch.Size([128, 12, 7, 1, 1])
After Layer 5: torch.Size([128, 2, 4, 1, 1])
Output Size:torch.Size([128, 2, 4, 1, 1])


## Experiment configuration

- Loss Function
- Optimizer
- Scheduler

In [8]:
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, weight_decay=0.0001, momentum=0, nesterov=False)
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.97)

num_epochs = 2

#### Run experiment

In [9]:
train(num_epochs, model, optimizer, scheduler, criterion, random_dataloader, device)

Running epoch : 0
Train Step
batch: 0, loss: 0.7134177088737488, correct : 0
batch: 1, loss: 0.7134486436843872, correct : 0
Total loss : 1.426866352558136
Total Acc : 0.0
Val Step
batch: 0, loss: 0.7133011817932129, correct : 0
batch: 1, loss: 0.7133904695510864, correct : 0
Total loss : 1.4266916513442993
Total Acc : 0.0
Running epoch : 1
Train Step
batch: 0, loss: 0.7133011817932129, correct : 0
batch: 1, loss: 0.7133321762084961, correct : 0
Total loss : 1.426633358001709
Total Acc : 0.0
Val Step
batch: 0, loss: 0.7131849527359009, correct : 0
batch: 1, loss: 0.7132735252380371, correct : 0
Total loss : 1.426458477973938
Total Acc : 0.0


In [ ]:

def train(num_epochs, model, optimizer, scheduler, criterion):
    
    epoch = 0
    
    while epoch < num_epochs:
        print(f'Running epoch : {epoch}')
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                
            else:
                model.eval()  # Set model to evaluate mode        
        
            running_loss = 0.0
            print(f'Running phase : {phase}')

            # Iterate over data.
            for i,_ in enumerate(range(2)):
                samples, targets = batch
                samples = samples.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):

                    preds = model(samples)
                    loss = criterion(preds, targets.float())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                print(f'batch: {i}, loss: {loss}')
                running_loss += loss.item()
        
            print(f'Total loss : {running_loss}')
            
        epoch += 1
    
